In [86]:
import xarray as xr
from dask.distributed import Client

In [87]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]
area = ((1-tmask) * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'depth', 'y':'gridY', 'x':'gridX'})

In [88]:
chunk_sizes = {'time': 1, 'depth': 40, 'gridY': 898, 'gridX': 398}

In [89]:
year = 2019
fname = 'SalishSeaCast_river_trace_day_avg_allR_00_40_{}0101_{}1231_e3t_3d_vol'.format(year, year)
volume = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_vol/{}.nc'.format(fname),chunks=chunk_sizes)

In [90]:
volume

<xarray.Dataset>
Dimensions:          (time: 365, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time             (time) datetime64[ns] 2019-01-01T12:00:00 ... 2019-12-31...
  * depth            (depth) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
  * gridY            (gridY) int64 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX            (gridX) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396 397
Data variables: (12/13)
    nooksack_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    skagit_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    snohomish_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    nisqually_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    elwha_river      (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    cowichan_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    ...               ...
    puntledge_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    salmon_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    homathko_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    squamish_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    fraser_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    other_rivers     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>

In [91]:
client=Client(processes=True, n_workers=4, local_directory='/tmp/cdonaldson')  # 4 is a good choice for n_workers even on salish, if I was the only person logged on or asked for space, could go as high as 16 but... be conscious of this
# threads_per_worker = 1 if needed, netcdf is not threadsafe now so be realiable by specifying 1 (at expense of just some efficiency)

In [92]:
meters = volume / area

In [93]:
for data_var in list(meters.data_vars):
    meters[data_var].attrs['units'] = 'm'

In [94]:
fname_save = fname[:-10] + 'meters_3d'
fname_save

'SalishSeaCast_river_trace_day_avg_allR_00_40_20190101_20191231_meters_3d'

In [95]:
meters.to_netcdf('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_meters/{}.nc'.format(fname_save), engine="netcdf4")

/home/cdonaldson/conda_envs/analysis-cassidy/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/cdonaldson/conda_envs/analysis-cassidy/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/cdonaldson/conda_envs/analysis-cassidy/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/cdonaldson/conda_envs/analysis-cassidy/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [96]:
client.close()